# Proyecto del Día 14 - Gestión Segura de Datos de Clientes

Este notebook implementa un sistema básico de gestión y análisis de datos de clientes, aplicando técnicas de pseudonimización, anonimización y balanceo de datos.


## Objetivo
El proyecto consiste en desarrollar un sistema de gestión y análisis de datos de clientes que implemente técnicas de pseudonimización, anonimización y balanceo de datos para proteger la privacidad de los usuarios y asegurar un análisis equitativo.


## Consignas del Proyecto
1. Cargar los datos de clientes provistos en un archivo CSV.
2. Aplicar técnicas de anonimización al campo `direccion`, `edad` y `salario` para proteger la identidad de los clientes.
3. Aplicar técnicas de pseudonimización al campo `nombre`.
4. Implementar una función de balanceo de datos para asegurar la representación equitativa de diversas categorías en el análisis posterior.
5. Realizar un análisis simple de los datos para identificar tendencias o patrones que podrían ser útiles para decisiones de negocios o marketing.


Este proyecto te permitirá aplicar prácticas esenciales de privacidad de datos y técnicas de manipulación de datos en un entorno práctico, preparándote para desafíos similares en entornos profesionales.

¡Mucha suerte y que te diviertas!

In [23]:
import os
import uuid

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.utils import resample

from dotenv import load_dotenv

In [4]:
load_dotenv('.env')

True

### 1. Cargar los datos de clientes provistos en un archivo CSV.

In [5]:
df: pd.DataFrame = pd.read_csv(os.environ['PROJECT_DATA'])

### 2. Aplicar técnicas de anonimización al campo `direccion`, `edad` y `salario` para proteger la identidad de los clientes.

In [ ]:
# Anonimizar el campo dirección por simple supresión
# (asumiendo que no es relevante para el estudio)
df.drop(['direccion'], axis=1, inplace=True)

In [ ]:
# Anonimizar el campo edad por redondeo o truncado
# adaptando las edades a sus respectivas décadas
df['edad'] = (df['edad'] // 10) * 10

In [8]:
# Anonimizar el campo salario con ruido aleatorio
noise: np.ndarray = np.random.normal(0, 10000, size=df['salario'].shape)
df['salario'] += noise

### 3. Aplicar técnicas de pseudonimización al campo `nombre`.

In [10]:
# Pseudonomización de los valores del campo nombre
# usando un identificador único universal (UUID) para cada valor
uuid_container: list[str] = []

for _ in range(len(df)):
    uuid_container.append(str(uuid.uuid4()))

df['uuid'] = uuid_container
df.drop(['nombre'], axis=1, inplace=True)

A partir de aquí, ya parece seguro mostrar algo de `df` para los siguientes pasos. 😁

In [11]:
df.head()

,edad,categoria,salario,uuid
0,60,0,74189.466539,1908635c-7c0a-4db0-82cc-bcd0971c4449
1,60,1,50776.658566,9d3178c7-c3e3-4def-9803-b8c799f0aa7b
2,10,0,66739.991400,14b51fca-4094-4dd8-a9f4-4e74b9d00184
3,20,1,23795.264394,e43c1f82-f14b-45ab-861e-705b0e8a2f0c
4,20,1,32557.826303,aaa3cb4c-f1d7-4593-831f-ad6c86982fb9


In [12]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200 entries, 0 to 199
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   edad       200 non-null    int64  
 1   categoria  200 non-null    int64  
 2   salario    200 non-null    float64
 3   uuid       200 non-null    object 
dtypes: float64(1), int64(2), object(1)
memory usage: 6.4+ KB


In [13]:
df.describe()

,edad,categoria,salario
count,200.00000,200.000000,200.000000
mean,37.70000,0.570000,48078.873231
std,15.74642,0.496318,15483.079063
min,10.00000,0.000000,6640.169101
25%,20.00000,0.000000,36517.018193
50%,40.00000,1.000000,48235.261437
75%,50.00000,1.000000,57318.668235
max,60.00000,1.000000,92882.610686


### 4. Implementar una función de balanceo de datos para asegurar la representación equitativa de diversas categorías en el análisis posterior.

Preliminarmente, los datos estadísticos de `df.describe()` parecen indicar que:

- Existen sólo dos categorías: `0` y `1`

- Hay más instancias para la segunda categoría que para la primera, tomando como referencia la media de `0.57`

No obstante, en honor a la rigurosidad, conviene verificar estos puntos antes de proceder con el objetivo de este paso.

In [ ]:
df['categoria'].unique()

array([0, 1], dtype=int64)

Efectivamente, existen sólo dos categorías: `0` y `1`. Veamos la proporción exacta de ellas en el conjunto `df`:

In [26]:
category_0: int = (df['categoria'] == 0).sum()
category_1: int = (df['categoria'] == 1).sum()

print(f'Instancias de categoría 0: {category_0}')
print(f'Instancias de categoría 1: {category_1}')

Instancias de categoría 0: 86
Instancias de categoría 1: 114


Efectivamente, se verifica que hay un desbalance en la repartición de instancias entre categorías en el conjunto de datos, en una razón de 86:114 para las categorías `0` y `1`, respectivamente.

Por lo tanto, procederemos con las operaciones para asegurar la representación equitativa de ambas categorías.

In [28]:
grouped_df = df.groupby('categoria')

balanced_data: pd.DataFrame = pd.DataFrame()

for name, group in grouped_df:
    balanced_group = resample(
        group,
        replace=True,
        n_samples=114,
        random_state=123
    )
    balanced_data = pd.concat([balanced_data, balanced_group])

category_0 = (balanced_data['categoria'] == 0).sum()
category_1 = (balanced_data['categoria'] == 1).sum()

print(f'Instancias de categoría 0: {category_0}')
print(f'Instancias de categoría 1: {category_1}')

Instancias de categoría 0: 114
Instancias de categoría 1: 114


Las instancias de ambas categorías se han igualado exitosamente. Veamos las características de `balanced_data`:

In [29]:
balanced_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 228 entries, 158 to 11
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   edad       228 non-null    int64  
 1   categoria  228 non-null    int64  
 2   salario    228 non-null    float64
 3   uuid       228 non-null    object 
dtypes: float64(1), int64(2), object(1)
memory usage: 8.9+ KB
